<h1>Initialisation</h1>

In [1]:
!pip install -q faiss-gpu "unstructured[docx]" llama-index langchain langchain-community langchain-huggingface docx2txt langchain-groq langchain-chroma

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.8 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4

In [2]:
import pandas as pd
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from llama_index.core import SimpleDirectoryReader
from langchain.docstore.document import Document as LangchainDocument
from tqdm.notebook import tqdm
import os
from langchain.vectorstores.chroma import Chroma 
import shutil
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import GPT2TokenizerFast
import re

In [3]:
DOCS_PATH = '/kaggle/input/3gpp-release18/rel18/rel18'
# Path to the directory to save Chroma database
CHROMA_PATH = "/kaggle/working/chroma"
EMBEDDING_MODEL_NAME = "BAAI/bge-small-en"

<h2>Vector DB creation</h2>

In [4]:
def chunking(RAW_KNOWLEDGE_BASE) : 
    tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
    def count_tokens(text: str)->int:
        return len(tokenizer.encode(text))
        
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,  # The maximum number of characters in a chunk: we selected this value arbitrarily
        chunk_overlap=24,  # The number of characters to overlap between chunks
        length_function = count_tokens
    )
    
    docs_processed = []
    for doc in tqdm(RAW_KNOWLEDGE_BASE):
        docs_processed += text_splitter.split_documents([doc])

    return docs_processed

In [5]:
def create_chunks(DOCS_PATH):
    def drop_first_pages(text:str) -> str:
        start_index = text.find("Foreword", 2000)
        if start_index != -1:
            return text[start_index:]
        else:
            return text
    documents = SimpleDirectoryReader(DOCS_PATH).load_data(num_workers=4)
    documents = [drop_first_pages(doc.text) for doc in documents]
    RAW_KNOWLEDGE_BASE = [
        LangchainDocument(page_content=doc)
        for doc in tqdm(documents)
    ]
    chunking_docs = chunking(RAW_KNOWLEDGE_BASE)
    return chunking_docs

chunked_docs = create_chunks(DOCS_PATH)

  0%|          | 0/554 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

  0%|          | 0/554 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1032 > 1024). Running this sequence through the model will result in indexing errors


<h2>Embedding model</h2>

In [6]:
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
def save_to_chroma(chunks: list[str], embedding_model):
  # Clear out the existing database directory if it exists
  if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

  # Create a new Chroma database from the documents using OpenAI embeddings
  db = Chroma.from_documents(
    docs_processed,
    embedding_model,
    persist_directory=CHROMA_PATH
  )

  # Persist the database to disk
  db.persist()
  print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

save_to_chroma(chunked_docs, embedding_model)

Chunks:   0%|          | 0/10 [00:00<?, ?it/s]

Chunks:   0%|          | 0/10 [00:00<?, ?it/s]

Chunks:   0%|          | 0/10 [00:00<?, ?it/s]

Saved 114099 chunks to /kaggle/working/chroma.


<ipython-input-16-73ede97f74b7>:20: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


<h1>llm and vector DB loading</h1>

In [7]:
from langchain_groq import ChatGroq
from kaggle_secrets import UserSecretsClient
import os

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GROQ_API_KEY")
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = secret_value_0
llm = ChatGroq(
    model="llama3-8b-8192", 
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
# Prepare the database
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_model)

<ipython-input-7-e5f7fe696c89>:17: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_model)


In [8]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
'{context}'
 - -
Given the context information, provide a correct answer to a multiple choice question on wireless communications and standards. 
Use only one option from 1, 2, 3, 4, or 5. Generate only the option number corresponding to the answer.
Question : {question}
Options : 
1) {option_1}
2) {option_2}
3) {option_3}
4) {option_4}
{option_5}

Answer: 
"""
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

<h2>Some useful finctions for test purpose</h2>

In [21]:
# Some usefull functions
def get_option_5(row: pd.Series) -> str:
    option_5 = row['option 5']
    if pd.isna(option_5):
        option_5 = ''
    else:
        option_5 = f'5) {option_5}'
    return option_5

def remove_release_number(question:str) -> str:
    question = re.findall('(.*?)(?:\s+\[3GPP Release \d+]|$)', question)[0]
    return question

def get_context(query_text, option:int):
    """
    Get context for query using retriever. Option is a int that tell the retriever to use.
    1 for chroma DB. 2 for Chroma DB with reranking. 3 for Hybrid Search. Else for None of them.
    """
    context_text = ""
    if option==1:
        # Retrieving the context from the DB using similarity search
        results = db.similarity_search_with_relevance_scores(query_text, k=3)
        # Combine context from matching documents
        context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])
        
    elif option==2:
        # Retrieving the context from the DB using similarity search
        results = db.similarity_search_with_relevance_scores(query_text, k=5)
        # Combine context from matching documents
        rerank_docs = co.rerank(
            model="rerank-english-v3.0",
            query=query_text, 
            documents=[doc.page_content for doc,_ in results], 
            top_n=3, 
            return_documents=True
        )
        # Extract reranked documents
        reranked_texts = [doc.document.text for doc in rerank_docs.results]
        context_text = "\n\n - -\n\n".join([doc for doc in reranked_texts])
        
    elif option==3:
        # Retrieving the context from the DB using similarity search
        results = compression_retriever.get_relevant_documents(query_text)
        # Extract reranked documents
        context_text = "\n\n - -\n\n".join([doc.page_content for doc in results])
    elif option==4:
        results=[]
        context_text = "No context is provided. Use prior knowledge to answer."
    
    return context_text

def format_row(row: pd.Series):
    option_5 = get_option_5(row)
    question = remove_release_number(row['question'])
    return question, row['option 1'], row['option 2'], row['option 3'], row['option 4'], option_5

def create_prompt(question: pd.Series, prompt_template, option:int):
    query, option_1, option_2, option_3, option_4, option_5 = format_row(question)
    context = get_context(query, option)
    prompt = prompt_template.format(
        context=context, 
        question=query,
        option_1=option_1,
        option_2=option_2,
        option_3=option_3,
        option_4=option_4,
        option_5=option_5
    )
    return prompt

def query_rag(question: pd.Series, prompt_template, llm, option:int):
    # Get the context from chunks
    prompt = create_prompt(question, prompt_template, option)
    # Generate response text based on the prompt
    response_text = llm.invoke(prompt)
    return response_text.content

def test_RAG(test_df, labels_df, option:int):
    answers = []
    wrong_answers = []
    for _, question in tqdm(test_df.iterrows(), total=test_df.shape[0]):
        answer = query_rag(question, prompt_template, llm, option)
        try:
            answer = int(answer)
            answers.append([question['Question_ID'], answer])
        except:
            resp = re.findall('\\n\\n(\d)', answer)[0]
            answers.append([question['Question_ID'], resp])
            wrong_answers.append([question['Question_ID'], answer])
        

    # Create a DataFrame with answers
    results = pd.DataFrame(answers, columns=['Question_ID', 'Prediction_ID'])    
    try:
        results['Question_ID'] = results['Question_ID'].astype('int')
        results['Prediction_ID'] = results['Prediction_ID'].astype('int')
        results_with_labels = pd.merge(results, labels_df,how='left',on='Question_ID')
        train_acc = 100 * (results_with_labels['Answer_ID'] == results_with_labels['Prediction_ID']).sum() / len(results_with_labels)
        return f'Test accuracy: {train_acc}%', wrong_answers
    except Exception as e:
        print(e)
        return f'Test accuracy: Erreur', wrong_answers

<h2>Test data loading</h2>

In [17]:
# Import test dataset
test = pd.read_json('/kaggle/input/3gpp-release18/rel18/train/TeleQnA_training.txt').T
test['Question_ID'] = test.index.str.split(' ').str[-1]
test_spec = test[test['category']=='Standards specifications']
test_overview = test[test['category']=='Standards overview']

labels_df = pd.read_csv('/kaggle/input/3gpp-release18/Q_A_ID_training.csv')
labels_df = labels_df[['Question_ID', 'Answer_ID']]

<h2>Without RAG TEST</h2>

In [23]:
# TEST without RAG : Standards Specifications
response, _ = test_RAG(test_spec.head(25), labels_df, option=4)
print(response)

  0%|          | 0/25 [00:00<?, ?it/s]

Test accuracy: 36.0%


In [24]:
# TEST without RAG : Standards Overview
response, _ = test_RAG(test_overview.head(25), labels_df, option=4)
print(response)

  0%|          | 0/25 [00:00<?, ?it/s]

Test accuracy: 52.0%


<h2>Simple RAG TEST</h2>

In [25]:
# TEST with simple RAG : Standards Specifications
response, _ = test_RAG(test_spec.head(25), labels_df, option=1)
print(response)

  0%|          | 0/25 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

...

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Test accuracy: 64.0%


In [26]:
# TEST with simple RAG : Standards Overview
response, _ = test_RAG(test_overview.head(25), labels_df, option=1)
print(response)

  0%|          | 0/25 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

...

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Test accuracy: 48.0%


<h1>RAG Optimization</h1>

In [27]:
!pip install -q cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.5/252.5 kB 5.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.1 MB/s eta 0:00:00a 0:00:01


<h2>Implementing Re-Ranking</h2>

In [28]:
import cohere

cohere_key = user_secrets.get_secret("COHERE_API_KEY")
# init client
co = cohere.Client(cohere_key)

<h2>Test ReRanking RAG</h2> 

In [29]:
# TEST with Reranking RAG : Standards Specifications
response, _ = test_RAG(test_spec.head(25), labels_df, option=2)
print(response)

  0%|          | 0/25 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

...

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Test accuracy: 60.0%


In [30]:
# TEST with Reranking RAG : Standards Overview
response, _ = test_RAG(test_overview.head(25), labels_df, option=2)
print(response)

  0%|          | 0/25 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

...

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Test accuracy: 56.0%


<h1>RAG optim : Implementing Hybrid Search with ensemble Retrieval</h1>

In [32]:
!pip install rank_bm25

In [33]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

retriever_vectordb = db.as_retriever(search_kwargs={"k": 5})
keyword_retriever = BM25Retriever.from_documents(chunked_docs)
keyword_retriever.k =  5
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb,keyword_retriever],weights=[0.6, 0.4])

In [34]:
import os
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_core.runnables import RunnablePassthrough

os.environ['COHERE_API_KEY'] = cohere_key
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

<ipython-input-34-c2461fe741c3>:7: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank()


<h2>Hybrid Search RAG TEST</h2>

In [35]:
# TEST with Hybrid search RAG : Standards Specifications
response, _ = test_RAG(test_spec.head(25), labels_df, option=3)
print(response)

  0%|          | 0/25 [00:00<?, ?it/s]

<ipython-input-21-626ed1fb76cf>:43: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = compression_retriever.get_relevant_documents(query_text)


Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

...

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Test accuracy: 76.0%


In [36]:
# TEST with Hybrid search RAG : Standards Overview
response, _ = test_RAG(test_overview.head(25), labels_df, option=3)
print(response)

  0%|          | 0/25 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

...

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Test accuracy: 80.0%
